# 05 — Task 4: Efficient Frontier & Optimal Portfolio (TSLA/BND/SPY)

In [ ]:
# === Colab Setup ===
!pip install -U pip --quiet
!pip install pandas numpy matplotlib seaborn scikit-learn statsmodels yfinance PyPortfolioOpt arch --quiet
import sys, os, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", os.getcwd())

In [ ]:
# === Configuration ===
START_DATE = "2015-07-01"
END_DATE   = "2025-07-31"
TICKERS    = ["TSLA", "BND", "SPY"]
BT_START   = "2024-08-01"
BT_END     = "2025-07-31"
TRADING_DAYS = 252
RISK_FREE_DAILY = 0.0

DATA_DIR = "data"
OUT_PLOTS = "outputs/plots"
OUT_MODELS = "outputs/models"
OUT_METRICS = "outputs/metrics"

for d in [DATA_DIR, OUT_PLOTS, OUT_MODELS, OUT_METRICS]:
    os.makedirs(d, exist_ok=True)


In [ ]:
# === Build expected returns & covariance ===
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pypfopt.efficient_frontier import EfficientFrontier

prices = pd.read_csv(f"{DATA_DIR}/prices_adj_close.csv", index_col=0, parse_dates=True)
returns = pd.read_csv(f"{DATA_DIR}/returns_daily.csv", index_col=0, parse_dates=True)
fc_series = pd.read_csv(f"{OUT_METRICS}/tsla_forecast_12m.csv", index_col=0, parse_dates=True).iloc[:,0]

# TSLA expected annual return from forecast
tsla_fc_daily = fc_series.pct_change().dropna()
tsla_exp_annual = (1 + tsla_fc_daily.mean())**TRADING_DAYS - 1

# BND & SPY historical annualized mean
hist_ann = (1 + returns[['BND','SPY']].mean())**TRADING_DAYS - 1

mu = pd.Series({"TSLA": tsla_exp_annual, "BND": hist_ann['BND'], "SPY": hist_ann['SPY']})
Sigma = returns[['TSLA','BND','SPY']].cov() * TRADING_DAYS
display("Expected returns (annualized)", mu, "Covariance (annualized)", Sigma)

ef = EfficientFrontier(mu, Sigma)
ef.max_sharpe(risk_free_rate=0.0)
w_ms = ef.clean_weights()
ret_ms, vol_ms, sharpe_ms = ef.portfolio_performance(verbose=False, risk_free_rate=0.0)

ef2 = EfficientFrontier(mu, Sigma)
ef2.min_volatility()
w_mv = ef2.clean_weights()
ret_mv, vol_mv, sharpe_mv = ef2.portfolio_performance(verbose=False, risk_free_rate=0.0)

print("Max Sharpe:", w_ms, (ret_ms, vol_ms, sharpe_ms))
print("Min Vol:", w_mv, (ret_mv, vol_mv, sharpe_mv))

# Save weights
import json
with open(f"{OUT_METRICS}/weights_max_sharpe.json","w") as f: json.dump(w_ms, f, indent=2)
with open(f"{OUT_METRICS}/weights_min_vol.json","w") as f: json.dump(w_mv, f, indent=2)


> Proceed to Notebook 06 to backtest the Max Sharpe (or Min Vol) weights against a 60/40 benchmark.